In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [6]:
data = pd.read_csv('universal_top_spotify_songs.csv')


In [2]:
# Вывод общей информации о данных
print("Общая информация о данных:")
print(f"Количество строк: {data.shape[0]}")
print(f"Количество столбцов: {data.shape[1]}")
print("\nТипы данных по столбцам:")
print(data.dtypes)

Общая информация о данных:
Количество строк: 10000
Количество столбцов: 20

Типы данных по столбцам:
spotify_id           object
daily_rank            int64
daily_movement        int64
weekly_movement       int64
country              object
popularity            int64
is_explicit            bool
duration_ms           int64
danceability        float64
energy              float64
key                   int64
loudness            float64
mode                  int64
speechiness         float64
acousticness        float64
instrumentalness    float64
liveness            float64
valence             float64
tempo               float64
time_signature        int64
dtype: object


In [7]:
from sklearn.model_selection import GridSearchCV
from catboost import CatBoostClassifier

# Предположим, что мы хотим предсказать страну песни
X = data.drop('country', axis=1)
y = data['country']
X = pd.get_dummies(X)

# Разделение данных на тренировочный и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Масштабирование признаков (нормализация)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


#### Метод CatBoost

Гиперпараметры:
   - depth: Этот параметр определяет глубину дерева. Увеличение значения этого параметра увеличивает сложность модели, что может привести к более точной подгонке к тренировочным данным, но также увеличивает риск переобучения.
   - learning_rate: Определяет скорость обучения модели. Этот параметр контролирует величину, на которую обновляются веса модели при каждой итерации. Более низкие значения могут способствовать стабильному обучению, но могут потребовать большего количества итераций для достижения оптимальной точности.
   - l2_leaf_reg: Этот параметр отвечает за коэффициент регуляризации L2 (Ridge). Он контролирует сложность модели, ограничивая веса и предотвращая переобучение. Большие значения могут уменьшить сложность модели, но могут также уменьшить её точность на тренировочных данных.

In [8]:
# Определение параметров для CatBoost
param_grid_catboost = {
    'depth': [1, 4, 7, 10],
    'learning_rate': [0.01, 0.1, 1],
    'l2_leaf_reg': [1, 3, 5, 9],
    'iterations': [100, 200,],
    'depth': [0, 3, 6],
    'loss_function': ['MultiClass', 'Logloss']
}

In [9]:
# Создание модели CatBoost
catboost = CatBoostClassifier()

# Подбор оптимальных параметров с помощью перекрестной проверки для CatBoost
grid_search_catboost = GridSearchCV(catboost, param_grid_catboost, refit=True, verbose=3, cv=5)
grid_search_catboost.fit(X_train_scaled, y_train)

# Получение лучших параметров для CatBoost
best_params_catboost = grid_search_catboost.best_params_
best_score_catboost = grid_search_catboost.best_score_

Fitting 5 folds for each of 144 candidates, totalling 720 fits
0:	learn: 2.9947742	total: 47ms	remaining: 4.65s
1:	learn: 2.9938361	total: 47.5ms	remaining: 2.33s
2:	learn: 2.9929173	total: 48.1ms	remaining: 1.55s
3:	learn: 2.9920173	total: 48.6ms	remaining: 1.17s
4:	learn: 2.9911356	total: 49.1ms	remaining: 933ms
5:	learn: 2.9902717	total: 49.7ms	remaining: 779ms
6:	learn: 2.9894250	total: 50.3ms	remaining: 668ms
7:	learn: 2.9885952	total: 50.8ms	remaining: 584ms
8:	learn: 2.9877817	total: 51.3ms	remaining: 519ms
9:	learn: 2.9869842	total: 52ms	remaining: 468ms
10:	learn: 2.9862021	total: 52.5ms	remaining: 425ms
11:	learn: 2.9854352	total: 53.2ms	remaining: 390ms
12:	learn: 2.9846830	total: 53.8ms	remaining: 360ms
13:	learn: 2.9839452	total: 54.3ms	remaining: 334ms
14:	learn: 2.9832213	total: 54.9ms	remaining: 311ms
15:	learn: 2.9825111	total: 55.4ms	remaining: 291ms
16:	learn: 2.9818142	total: 55.8ms	remaining: 272ms
17:	learn: 2.9811303	total: 56ms	remaining: 255ms
18:	learn: 2.9804

/home/dmitrii/.local/lib/python3.10/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(


0:	learn: 2.9947817	total: 399us	remaining: 39.5ms
1:	learn: 2.9938509	total: 847us	remaining: 41.5ms
2:	learn: 2.9929394	total: 1.17ms	remaining: 37.7ms
3:	learn: 2.9920465	total: 1.47ms	remaining: 35.4ms
4:	learn: 2.9911717	total: 1.79ms	remaining: 33.9ms
5:	learn: 2.9903146	total: 2.1ms	remaining: 32.9ms
6:	learn: 2.9894746	total: 2.41ms	remaining: 32ms
7:	learn: 2.9886514	total: 2.8ms	remaining: 32.2ms
8:	learn: 2.9878443	total: 3.1ms	remaining: 31.3ms
9:	learn: 2.9870531	total: 3.39ms	remaining: 30.5ms
10:	learn: 2.9862773	total: 3.68ms	remaining: 29.8ms
11:	learn: 2.9855165	total: 3.99ms	remaining: 29.2ms
12:	learn: 2.9847703	total: 4.28ms	remaining: 28.7ms
13:	learn: 2.9840383	total: 4.6ms	remaining: 28.2ms
14:	learn: 2.9833203	total: 4.89ms	remaining: 27.7ms
15:	learn: 2.9826157	total: 5.18ms	remaining: 27.2ms
16:	learn: 2.9819244	total: 5.49ms	remaining: 26.8ms
17:	learn: 2.9812459	total: 5.8ms	remaining: 26.4ms
18:	learn: 2.9805799	total: 6.1ms	remaining: 26ms
19:	learn: 2.97

0:	learn: 2.9947787	total: 329us	remaining: 32.6ms
1:	learn: 2.9938449	total: 703us	remaining: 34.5ms
2:	learn: 2.9929303	total: 1.01ms	remaining: 32.7ms
3:	learn: 2.9920343	total: 1.37ms	remaining: 32.9ms
4:	learn: 2.9911565	total: 1.66ms	remaining: 31.6ms
5:	learn: 2.9902964	total: 2ms	remaining: 31.3ms
6:	learn: 2.9894535	total: 2.38ms	remaining: 31.7ms
7:	learn: 2.9886272	total: 2.7ms	remaining: 31ms
8:	learn: 2.9878173	total: 2.99ms	remaining: 30.2ms
9:	learn: 2.9870231	total: 3.34ms	remaining: 30.1ms
10:	learn: 2.9862444	total: 3.63ms	remaining: 29.3ms
11:	learn: 2.9854807	total: 3.94ms	remaining: 28.9ms
12:	learn: 2.9847316	total: 4.28ms	remaining: 28.7ms
13:	learn: 2.9839968	total: 4.59ms	remaining: 28.2ms
14:	learn: 2.9832758	total: 4.9ms	remaining: 27.8ms
15:	learn: 2.9825685	total: 5.25ms	remaining: 27.5ms
16:	learn: 2.9818743	total: 5.56ms	remaining: 27.1ms
17:	learn: 2.9811931	total: 5.87ms	remaining: 26.7ms
18:	learn: 2.9805245	total: 6.21ms	remaining: 26.5ms
19:	learn: 2

[CV 1/5] END depth=0, iterations=100, l2_leaf_reg=1, learning_rate=0.01, loss_function=Logloss;, score=nan total time=   0.1s
[CV 2/5] END depth=0, iterations=100, l2_leaf_reg=1, learning_rate=0.01, loss_function=Logloss;, score=nan total time=   0.1s
[CV 3/5] END depth=0, iterations=100, l2_leaf_reg=1, learning_rate=0.01, loss_function=Logloss;, score=nan total time=   0.1s
[CV 4/5] END depth=0, iterations=100, l2_leaf_reg=1, learning_rate=0.01, loss_function=Logloss;, score=nan total time=   0.1s
[CV 5/5] END depth=0, iterations=100, l2_leaf_reg=1, learning_rate=0.01, loss_function=Logloss;, score=nan total time=   0.1s
0:	learn: 2.9865746	total: 395us	remaining: 39.1ms
1:	learn: 2.9791964	total: 758us	remaining: 37.2ms
2:	learn: 2.9731484	total: 1.1ms	remaining: 35.7ms
3:	learn: 2.9681267	total: 1.42ms	remaining: 34.1ms
4:	learn: 2.9639160	total: 1.74ms	remaining: 33.1ms
5:	learn: 2.9603582	total: 2.05ms	remaining: 32.2ms
6:	learn: 2.9573339	total: 2.36ms	remaining: 31.4ms
7:	learn:

...

107:	learn: 0.9654545	total: 1.36s	remaining: 1.16s
108:	learn: 0.9627542	total: 1.38s	remaining: 1.15s
109:	learn: 0.9618693	total: 1.39s	remaining: 1.14s
110:	learn: 0.9607244	total: 1.4s	remaining: 1.12s
111:	learn: 0.9596806	total: 1.42s	remaining: 1.11s
112:	learn: 0.9586585	total: 1.43s	remaining: 1.1s
113:	learn: 0.9576824	total: 1.44s	remaining: 1.09s
114:	learn: 0.9568429	total: 1.45s	remaining: 1.07s
115:	learn: 0.9558110	total: 1.47s	remaining: 1.06s
116:	learn: 0.9522821	total: 1.48s	remaining: 1.05s
117:	learn: 0.9511328	total: 1.49s	remaining: 1.03s
118:	learn: 0.9408780	total: 1.5s	remaining: 1.02s
119:	learn: 0.9394854	total: 1.51s	remaining: 1.01s
120:	learn: 0.9387314	total: 1.53s	remaining: 997ms
121:	learn: 0.9377649	total: 1.54s	remaining: 984ms
122:	learn: 0.9368895	total: 1.55s	remaining: 972ms
123:	learn: 0.9357677	total: 1.56s	remaining: 959ms
124:	learn: 0.9347241	total: 1.58s	remaining: 946ms
125:	learn: 0.9317534	total: 1.59s	remaining: 934ms
126:	learn: 0.9

64:	learn: 1.2351723	total: 838ms	remaining: 1.74s
65:	learn: 1.2340812	total: 854ms	remaining: 1.73s
66:	learn: 1.2323226	total: 867ms	remaining: 1.72s
67:	learn: 1.2218189	total: 880ms	remaining: 1.71s
68:	learn: 1.2206761	total: 891ms	remaining: 1.69s
69:	learn: 1.2198392	total: 903ms	remaining: 1.68s
70:	learn: 1.2106710	total: 916ms	remaining: 1.66s
71:	learn: 1.2058742	total: 928ms	remaining: 1.65s
72:	learn: 1.2049222	total: 941ms	remaining: 1.64s
73:	learn: 1.2038872	total: 954ms	remaining: 1.62s
74:	learn: 1.2019785	total: 966ms	remaining: 1.61s
75:	learn: 1.2004568	total: 980ms	remaining: 1.6s
76:	learn: 1.1992898	total: 993ms	remaining: 1.59s
77:	learn: 1.1980008	total: 1s	remaining: 1.57s
78:	learn: 1.1968710	total: 1.02s	remaining: 1.56s
79:	learn: 1.1953653	total: 1.03s	remaining: 1.55s
80:	learn: 1.1941188	total: 1.04s	remaining: 1.53s
81:	learn: 1.1930852	total: 1.06s	remaining: 1.52s
82:	learn: 1.1917095	total: 1.07s	remaining: 1.51s
83:	learn: 1.1643095	total: 1.08s	r

/home/dmitrii/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
360 fits failed out of a total of 720.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
360 fits failed with the following error:
Traceback (most recent call last):
  File "/home/dmitrii/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/dmitrii/.local/lib/python3.10/site-packages/catboost/core.py", line 5100, in fit
    self._fit(X, y, cat_features, text_features, embedding_features, None, sample_weight, None, None, None, None, baseline, use_best_model,
  File "/home/dmitrii/.local/lib/python3.10/site-packages/

0:	learn: 2.9707021	total: 12.1ms	remaining: 2.4s
1:	learn: 2.9484058	total: 27ms	remaining: 2.67s
2:	learn: 2.9309850	total: 40.4ms	remaining: 2.65s
3:	learn: 2.9065309	total: 53.2ms	remaining: 2.61s
4:	learn: 2.8784372	total: 65.6ms	remaining: 2.56s
5:	learn: 2.8540947	total: 78.2ms	remaining: 2.53s
6:	learn: 2.8249020	total: 91.1ms	remaining: 2.51s
7:	learn: 2.8016445	total: 104ms	remaining: 2.5s
8:	learn: 2.7788416	total: 117ms	remaining: 2.47s
9:	learn: 2.7613061	total: 130ms	remaining: 2.46s
10:	learn: 2.7458694	total: 143ms	remaining: 2.45s
11:	learn: 2.7284730	total: 155ms	remaining: 2.43s
12:	learn: 2.7141738	total: 168ms	remaining: 2.42s
13:	learn: 2.7001773	total: 181ms	remaining: 2.4s
14:	learn: 2.6843457	total: 194ms	remaining: 2.39s
15:	learn: 2.6721417	total: 210ms	remaining: 2.41s
16:	learn: 2.6557412	total: 223ms	remaining: 2.4s
17:	learn: 2.6362848	total: 240ms	remaining: 2.43s
18:	learn: 2.6132688	total: 252ms	remaining: 2.4s
19:	learn: 2.5976139	total: 265ms	remaini

175:	learn: 1.5451003	total: 2.31s	remaining: 316ms
176:	learn: 1.5449626	total: 2.33s	remaining: 302ms
177:	learn: 1.5380877	total: 2.34s	remaining: 290ms
178:	learn: 1.5362029	total: 2.36s	remaining: 276ms
179:	learn: 1.5360754	total: 2.37s	remaining: 263ms
180:	learn: 1.5359569	total: 2.38s	remaining: 250ms
181:	learn: 1.5304166	total: 2.4s	remaining: 237ms
182:	learn: 1.5301896	total: 2.41s	remaining: 224ms
183:	learn: 1.5300499	total: 2.42s	remaining: 211ms
184:	learn: 1.5287940	total: 2.43s	remaining: 197ms
185:	learn: 1.5286542	total: 2.45s	remaining: 184ms
186:	learn: 1.5249705	total: 2.46s	remaining: 171ms
187:	learn: 1.5248904	total: 2.47s	remaining: 158ms
188:	learn: 1.5233044	total: 2.48s	remaining: 145ms
189:	learn: 1.5228481	total: 2.5s	remaining: 131ms
190:	learn: 1.5227002	total: 2.51s	remaining: 118ms
191:	learn: 1.5225283	total: 2.52s	remaining: 105ms
192:	learn: 1.5224363	total: 2.54s	remaining: 92.1ms
193:	learn: 1.5223219	total: 2.55s	remaining: 78.9ms
194:	learn: 

In [10]:
test_score = grid_search_catboost.score(X_test_scaled, y_test)
print("Лучшие параметры для CatBoost:", best_params_catboost)
print("Лучший результат (точность) для CatBoost:", best_score_catboost)
print("Точность на тестовом наборе данных:", test_score)

best_model = grid_search_catboost.best_estimator_
predictions = best_model.predict(X_test_scaled)

# Отчет по классификации
print("\nОтчет по классификации:")
print(classification_report(y_test, predictions))

Лучшие параметры для CatBoost: {'depth': 6, 'iterations': 200, 'l2_leaf_reg': 9, 'learning_rate': 0.1, 'loss_function': 'MultiClass'}
Лучший результат (точность) для CatBoost: 0.34498447204968946
Точность на тестовом наборе данных: 0.32338308457711445

Отчет по классификации:
              precision    recall  f1-score   support

          PK       0.38      0.38      0.38         8
          PL       1.00      0.13      0.24        15
          PT       0.40      0.29      0.33         7
          PY       0.14      0.10      0.12        10
          RO       0.75      0.60      0.67        10
          SA       0.40      0.18      0.25        11
          SE       0.00      0.00      0.00         5
          SG       0.14      0.11      0.12         9
          SK       0.67      0.60      0.63        10
          SV       0.17      0.06      0.09        17
          TH       0.11      0.11      0.11         9
          TR       0.45      0.56      0.50         9
          TW       0